In [1]:
import warnings
from tqdm import tqdm
import numpy as np
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
import catboost as cab
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,precision_recall_fscore_support,roc_curve,auc,roc_auc_score
from sklearn.model_selection import GridSearchCV
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import gc
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

#from featexp import get_univariate_plots#用于特征筛选，需要先安装featexp
warnings.filterwarnings("ignore")
plt.rcParams['font.sans-serif']=['Simhei']
plt.rcParams['axes.unicode_minus']=False
import json
import jieba
from datetime import datetime

C:\ProgramData\Anaconda3\lib\site-packages\dask\dataframe\utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
entprise_train = pd.read_csv('train/entprise_info.csv',engine='python')
base_info = pd.read_csv('train/base_info.csv',engine='python',encoding='utf-8')
entprise_test = pd.read_csv('entprise_evaluate.csv')
reccap_predict = pd.read_csv('reccap_predict.csv')['reccap']
empnum_predict = pd.read_csv('empnum_predict.csv')['empnum']

In [ ]:
base_info=pd.read_csv('train/base_info.csv')#企业的基本信息
annual_report_info=pd.read_csv('train/annual_report_info.csv')#企业的年报基本信息
tax_info=pd.read_csv('train/tax_info.csv')#企业的纳税信息
change_info=pd.read_csv('train/change_info.csv')#变更信息
news_info=pd.read_csv('train/news_info.csv')#舆情信18
other_info=pd.read_csv('train/other_info.csv')#其它信息
entprise_info=pd.read_csv('train/entprise_info.csv')#企业标注信息{0: 13884, 1: 981}
entprise_evaluate=pd.read_csv('entprise_evaluate.csv')#未标注信息

In [ ]:
#change_info
change_info_clean = change_info.groupby('id',sort=False).agg('mean')
change_info['bgrq'] = (change_info['bgrq'] / 10000000000).astype(int)
change_info_df = change_info.groupby('id').agg({
    'bgxmdm': ['count', 'nunique'],
    'bgq':['nunique'],
    'bgh': ['nunique'],
    'bgrq': ['nunique'],
})
change_info_df.columns = pd.Index(['changeinfo_' + e[0] + "_" + e[1].upper() 
                                for e in change_info_df.columns.tolist()])
change_info_df = change_info_df.reset_index()
change_info = pd.merge(change_info_df,change_info_clean,how='left',on='id')
#other_info
#空值大于0.5的列都删除掉
other_info = other_info[~other_info['id'].duplicated()]
other_info['other_SUM'] = other_info[['legal_judgment_num', 'brand_num', 'patent_num']].sum(1)
other_info['other_NULL_SUM'] = other_info[['legal_judgment_num', 'brand_num', 'patent_num']].isnull().astype(int).sum(1)
#news_info
news_info_clean=news_info.drop(['public_date'],axis=1)
#对object类型进行编码
news_info_clean['positive_negtive']=news_info_clean['positive_negtive'].fillna("中立")
#
dic={}
cate=news_info_clean.positive_negtive.unique()
for i in range(len(cate)):
    dic[cate[i]]=i
#
news_info_clean['positive_negtive']=news_info_clean['positive_negtive'].map(dic)
news_info_clean = news_info_clean.groupby('id',sort=False).agg('mean')
news_info_clean=pd.DataFrame(news_info_clean).reset_index()
news_info['public_date'] = news_info['public_date'].apply(lambda x: x if '-' in str(x) else np.nan)
news_info['public_date'] = pd.to_datetime(news_info['public_date'])
news_info['public_date'] = (datetime.now() - news_info['public_date']).dt.days

news_info_df = news_info.groupby('id').agg({'public_date': ['count','max','min','mean']}).reset_index()
news_info_df.columns = ['id', 'public_date_COUNT', 'public_MAX', 'public_MIN', 'public_MEAN']
news_info_df2 = pd.pivot_table(news_info, index='id', columns='positive_negtive', aggfunc='count').reset_index()
news_info_df2.columns = ['id', 'news_COUNT1', 'news_COUNT2', 'news_COUNT3']
news_info_df = pd.merge(news_info_df, news_info_df2)
news_info_1 = pd.merge(news_info_df,news_info_clean,how='left')
#处理annual_report_info的数据
#空值大于0.5的列都删除掉
annual_report_info_clean=annual_report_info.dropna(thresh=annual_report_info.shape[0]*0.5,how='all',axis=1)
#对object类型进行编码
annual_report_info_clean['BUSSTNAME']=annual_report_info_clean['BUSSTNAME'].fillna("无")
dic = {'无':-1,'开业':0, '歇业':1, '停业':2, '清算':3}
#
annual_report_info_clean['BUSSTNAME']=annual_report_info_clean['BUSSTNAME'].map(dic)
annual_report_info_clean = annual_report_info_clean.groupby('id',sort=False).agg('mean')
annual_report_info_clean=pd.DataFrame(annual_report_info_clean).reset_index()
#处理tax数据
tax_info_clean=tax_info.copy()
tax_info_clean['START_DATE']=pd.to_datetime(tax_info_clean['START_DATE'])
tax_info_clean['END_DATE']=pd.to_datetime(tax_info_clean['END_DATE'])
tax_info_clean['gap_day']=(tax_info_clean['END_DATE']-tax_info_clean['START_DATE']).dt.total_seconds()//3600//24
tax_info_clean=tax_info_clean.drop(['START_DATE','END_DATE'],axis=1)
tax_info_clean['TAX_CATEGORIES']=tax_info_clean['TAX_CATEGORIES'].fillna("无")#17 unique
tax_info_clean['TAX_ITEMS']=tax_info_clean['TAX_ITEMS'].fillna("无")#275 TAX_ITEMS
#对object类型进行编码
dic={}
cate=tax_info_clean.TAX_CATEGORIES.unique()
for i in range(len(cate)):
    dic[cate[i]]=i
tax_info_clean['TAX_CATEGORIES']=tax_info_clean['TAX_CATEGORIES'].map(dic)
#
dic={}
cate=tax_info_clean.TAX_ITEMS.unique()
for i in range(len(cate)):
    dic[cate[i]]=i
tax_info_clean['TAX_ITEMS']=tax_info_clean['TAX_ITEMS'].map(dic)
tax_info_clean['income']=tax_info_clean['TAX_AMOUNT']/tax_info_clean['TAX_RATE']
#
tax_info_clean = tax_info_clean.groupby('id',sort=False).agg('mean')
tax_info_clean=pd.DataFrame(tax_info_clean).reset_index()
#税额分箱
tax_info_clean['TAX_AMOUNT']=tax_info_clean['TAX_AMOUNT'].fillna(tax_info_clean['TAX_AMOUNT'].median())
tax_info_clean['bucket_TAX_AMOUNT']=pd.qcut(tax_info_clean['TAX_AMOUNT'], 10, labels=False,duplicates='drop')
print('finished .............')

In [ ]:
news_info_1

In [3]:
entprise = pd.concat([entprise_train,entprise_test])
data = pd.merge(entprise,base_info,on='id')
data.drop(['ptbusscope','midpreindcode','score'],axis=1,inplace=True)

In [ ]:
data=data.merge(annual_report_info_clean,how='outer')
data=data.merge(tax_info_clean,how='outer')
data=data.merge(change_info,how='outer')
data=data.merge(news_info_1,how='outer',on='id')
data=data.merge(other_info,how='outer')

In [ ]:
data.shape

In [4]:
data['enttypeitem'] = data['enttypegb']//10*10
data['enttypeminu'] = data['enttypegb']
data['regcap'] = data['regcap'].fillna(data['regcap'].mean())
data['reccap'] = reccap_predict
data['empnum'] = empnum_predict
data['reccap'] = data['reccap']
data['opform'].fillna(10,inplace=True)
data['opform'] = data['opform'].astype(str)
from sklearn.preprocessing import LabelEncoder
labelencoder1 = LabelEncoder()
labelencoder2 = LabelEncoder()
labelencoder3 = LabelEncoder()
labelencoder4 = LabelEncoder()
data['opform'] = labelencoder1.fit_transform(data['opform'])
data['oploc'] = labelencoder2.fit_transform(data['oploc'])
data['industryphy'] = labelencoder3.fit_transform(data['industryphy'])
data['oplocdistrict'] = labelencoder4.fit_transform(data['oplocdistrict'])
def opto_process(x):
    if len(x)>3:
        return x[:4]
    else:
        return x
data['opfrom'] = data['opfrom'].map(lambda x:x[:4])
data['opto'].fillna('nan',inplace=True)
data['opto'] = data['opto'].map(opto_process)
data['opfrom'] = data['opfrom'].astype(int)
data['opto'] = data['opto'].astype(float)
data['industryco'] = data['industryco'].fillna(data['industryco'].mean())
data.loc[data['opto'].isnull(),"opto"] = data[data['opto'].isnull()]['opfrom']+50.0
data['opscope'] = data['opscope'].apply(lambda x:x.replace("、","/").replace("；","/").replace("，","/").replace("。","/"))
data['opscope'] = data['opscope'].apply(lambda x:x.replace("（","/").replace("）","/").replace("，","/").replace("。","/"))
data['opscope_1'] = data['opscope'].apply(lambda x:x.split("/"))
data['opscope_feature'] = data['opscope'].apply(lambda x:x.split("/")[0][-2:])
le2 = LabelEncoder()
data['opscope_feature'] = le2.fit_transform(data['opscope_feature'])

In [ ]:
# data['enttypeitem_RANK'] = data['enttypeitem'].map(data.groupby(['enttypeitem'])['label'].mean().rank())
# data['enttypeminu_RANK'] = data['enttypeminu'].map(data.groupby(['enttypeminu'])['label'].mean().rank())
# data['enttype_RANK'] = data['enttype'].map(data.groupby(['enttype'])['label'].mean().rank())

In [5]:
def bucket(name,bucket_len):
    gap_list=[data[name].quantile(i/bucket_len) for i in range(bucket_len+1)]
    len_data=len(data[name])
    new_col=[]
    for i in data[name].values:
        for j in range(len(gap_list)):
            if gap_list[j]>=i:
                encode=j
                break
        new_col.append(encode)
    return new_col
#注册资本_实缴资本
data['regcap_reccap']=data['regcap']-data['reccap']
#注册资本分桶
data['regcap']=data['regcap'].fillna(data['regcap'].median())
data['bucket_regcap']=bucket('regcap',5)
#实缴资本分桶
data['reccap']=data['reccap'].fillna(data['reccap'].median())
data['bucket_reccap']=bucket('reccap',5)
#注册资本_实缴资本分桶
data['regcap_reccap']=data['regcap_reccap'].fillna(data['regcap_reccap'].median())
data['bucket_regcap_reccap']=bucket('regcap_reccap',5)
print('分桶完毕.................')

分桶完毕.................


In [6]:
# data['dom'] = data['dom'].apply(lambda x:x[:16])
le7 = LabelEncoder()
data['dom'] = le7.fit_transform(data['dom'])

In [7]:
from tqdm import tqdm
def cross_two(name_1,name_2):
    new_col=[]
    encode=0
    dic={}
    val_1=data[name_1]
    val_2=data[name_2]
    for i in tqdm(range(len(val_1))):
        tmp=str(val_1[i])+'_'+str(val_2[i])
        if tmp in dic:
            new_col.append(dic[tmp])
        else:
            dic[tmp]=encode
            new_col.append(encode)
            encode+=1
    return new_col
#作企业类型-小类的交叉特征
data['enttypegb']=data['enttypegb'].fillna("无")
data['enttypeitem']=data['enttypeitem'].fillna("无")
new_col=cross_two('enttypegb','enttypeitem')#作企业类型-小类的交叉特征
data['enttypegb_enttypeitem']=new_col
new_col=cross_two('enttype','enttypegb_enttypeitem')#作企业类型-小类的交叉特征
data['enttype_enttypegb_enttypeitem']=new_col
#
#行业类别-细类的交叉特征
data['industryphy']=data['industryphy'].fillna("无")
data['industryco']=data['industryco'].fillna("无")
new_col=cross_two('industryphy','industryco')#作企业类型-小类的交叉特征
data['industryphy_industryco']=new_col
#企业类型-行业类别的交叉特征
new_col=cross_two('enttypegb','industryphy')#作企业类型-小类的交叉特征
data['enttypegb_industryphy']=new_col
#行业类别-企业类型小类的交叉特征
new_col=cross_two('industryphy','enttypeitem')#作企业类型-小类的交叉特征
data['industryphy_enttypeitem']=new_col
#行业类别细类--企业类型小类的交叉特征
new_col=cross_two('industryco','enttypeitem')#作企业类型-小类的交叉特征
data['industryco_enttypeitem']=new_col
#企业类型-小类-行业类别-细类的交叉特征
# new_col=cross_two('enttypegb_enttypeitem','industryphy_industryco')#作企业类型-小类的交叉特征
# data['enttypegb_enttypeitem_industryphy_industryco']=new_col

100%|████████████████████████████████████████████████████████████████████████| 24865/24865 [00:00<00:00, 132847.12it/s]


In [8]:
for fea in ['industryphy','enttype','enttypeitem','enttypeminu']:
    data[fea+'_counts'] = data.groupby([fea])['id'].transform('count')

In [9]:
x_train = data[data['label'].notnull()]
kind = x_train['label']
x_test = data[data['label'].isnull()]
features_name = [i for i in data.columns if i not in ['id','positive_negtive','id_feature','regcap','reccap','enttype','enttypeitem','enttypeminu','bucket_regcap_reccap','townsign','count_opscope','opscope_1','oplocdistrict','adbusign','compform','opform','orgid','exenum','opscope','enttypegb','label','state','jobid','regtype','parnum','venind','protype','oploc','forreccap','forregcap','congro']]
#features_name = ['tfidif_opscope', 'industryphy', 'industryco', 'dom', 'opfrom', 'opto', 'empnum', 'enttpe_year', 'opscope_length', 'opscope_label', 'regcap_reccap', 'bucket_regcap', 'bucket_reccap', 'enttypegb_enttypeitem', 'enttype_enttypegb_enttypeitem', 'industryphy_industryco', 'industryco_enttypeitem', 'enttypegb_enttypeitem_industryphy_industryco', 'industryphy_counts', 'enttype_counts', 'enttypeitem_counts', 'empnum_counts', 'opfrom_counts', 'regcap_counts', 'reccap_counts', 'opto_counts', 'enttypeminu_counts']
#features_name = ['industryphy', 'industryco', 'dom', 'opfrom', 'opto', 'empnum', 'enttpe_year', 'opscope_length', 'opscope_label', 'regcap_reccap', 'bucket_regcap', 'bucket_reccap', 'enttypegb_enttypeitem', 'industryphy_industryco', 'enttypegb_industryphy', 'industryphy_enttypeitem', 'industryco_enttypeitem', 'enttypegb_enttypeitem_industryphy_industryco', 'industryphy_counts', 'enttype_counts', 'enttypeitem_counts', 'empnum_counts', 'opfrom_counts', 'regcap_counts', 'reccap_counts', 'opto_counts', 'enttypeminu_counts', 'id_feature_dom']
train_data = x_train[features_name]
test_data = x_test[features_name]
# train_data = train_data.fillna(-1)
# test_data = test_data.fillna(-1)

In [10]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14865 entries, 0 to 14864
Data columns (total 20 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   industryphy                    14865 non-null  int64  
 1   industryco                     14865 non-null  float64
 2   dom                            14865 non-null  int64  
 3   opfrom                         14865 non-null  int32  
 4   opto                           14865 non-null  float64
 5   empnum                         14865 non-null  float64
 6   opscope_feature                14865 non-null  int64  
 7   regcap_reccap                  14865 non-null  float64
 8   bucket_regcap                  14865 non-null  int64  
 9   bucket_reccap                  14865 non-null  int64  
 10  enttypegb_enttypeitem          14865 non-null  int64  
 11  enttype_enttypegb_enttypeitem  14865 non-null  int64  
 12  industryphy_industryco         14865 non-null 

In [11]:
def eval_score(y_test,y_pre):
    _,_,f_class,_=precision_recall_fscore_support(y_true=y_test,y_pred=y_pre,labels=[0,1],average=None)
    fper_class={'合法':f_class[0],'违法':f_class[1],'f1':f1_score(y_test,y_pre)}
    return fper_class
#
def k_fold_serachParmaters(model,train_val_data,train_val_kind):
    mean_f1=0
    mean_f1Train=0
    n_splits=5
    sk = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=2020)
    for train, test in sk.split(train_val_data, train_val_kind):
        x_train = train_val_data.iloc[train]
        y_train = train_val_kind.iloc[train]
        x_test = train_val_data.iloc[test]
        y_test = train_val_kind.iloc[test]

        model.fit(x_train, y_train)
        pred = model.predict(x_test)
        fper_class =  eval_score(y_test,pred)
        mean_f1+=fper_class['f1']/n_splits
        #print(fper_class)
        pred_Train = model.predict(x_train)
        fper_class_train =  eval_score(y_train,pred_Train)
        mean_f1Train+=fper_class_train['f1']/n_splits
    #print('mean valf1:',mean_f1)
    #print('mean trainf1:',mean_f1Train)
    return mean_f1

In [14]:
xlf=xgb.XGBClassifier(max_depth=5
                      ,learning_rate=0.05
                      ,n_estimators=150
                      ,reg_alpha=0.01
                      ,n_jobs=5
                      ,importance_type='total_cover'
                     )
#
llf=lgb.LGBMClassifier(num_leaves=10
                           ,max_depth=5
                           ,min_child_samples=10
                           ,learning_rate=0.03
                           ,n_estimators=150
                           ,n_jobs=-1
                           )
  
clf=cab.CatBoostClassifier(iterations=500
                              ,learning_rate=0.01
                              ,depth=4
                              ,l2_leaf_reg=3
                              ,silent=True
                              ,thread_count=8
                              ,task_type='CPU'
                              )

rf = RandomForestClassifier(oob_score=True, random_state=2020,
            n_estimators= 60,max_depth=13,min_samples_split=10)
#k_fold_serachParmaters(rf,train_data,kind)
print('xlf:',k_fold_serachParmaters(xlf,train_data,kind))
print('llf:',k_fold_serachParmaters(llf,train_data,kind))
print('clf:',k_fold_serachParmaters(clf,train_data,kind)) 
print('rf:',k_fold_serachParmaters(rf,train_data,kind))

xlf: 0.840379639061893
llf: 0.841320641054037
clf: 0.8453457117327261
rf: 0.8466039186976277


In [15]:
details = []
answers = []
mean_f1=0
n_splits=5
sk = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=2020)
cnt=0
for train, test in sk.split(train_data, kind):
    x_train = train_data.iloc[train]
    y_train = kind.iloc[train]
    x_test = train_data.iloc[test]
    y_test = kind.iloc[test]

    xlf.fit(x_train, y_train)
    pred_xgb = xlf.predict(x_test)
    weight_xgb = eval_score(y_test,pred_xgb)['f1']

    llf.fit(x_train, y_train)
    pred_llf = llf.predict(x_test)
    weight_lgb = eval_score(y_test,pred_llf)['f1']

    clf.fit(x_train, y_train)
    pred_cab = clf.predict(x_test)
    weight_cab =  eval_score(y_test,pred_cab)['f1']

    rf.fit(x_train, y_train)
    pred_rf = rf.predict(x_test)
    weight_rf =  eval_score(y_test,pred_rf)['f1']


    prob_xgb = xlf.predict_proba(x_test)
    prob_lgb = llf.predict_proba(x_test)
    prob_cab = clf.predict_proba(x_test)
    prob_rf = rf.predict_proba(x_test)

    scores = []
    ijkl = []
    weight = np.arange(0.1, 1.05, 0.1)
    for i, item1 in enumerate(weight):
        for j, item2 in enumerate(weight[weight <= (1 - item1)]):
            for k, item3 in enumerate(weight[weight <= (1 - item1-item2)]):
                prob_end = prob_xgb * item1 + prob_lgb * item2 + prob_cab *item3+prob_rf*(1 - item1 - item2-item3)
                #prob_end = np.sqrt(prob_xgb**2 * item1 + prob_lgb**2 * item2 + prob_cab**2 *item3+prob_rf**2*(1 - item1 - item2-item3))
                score = eval_score(y_test,np.argmax(prob_end,axis=1))['f1']
                scores.append(score)
                ijkl.append((item1, item2,item3, 1 - item1 - item2-item3))
    ii = ijkl[np.argmax(scores)][0]
    jj = ijkl[np.argmax(scores)][1]
    kk = ijkl[np.argmax(scores)][2]
    ll = ijkl[np.argmax(scores)][3]

    details.append(max(scores))
    details.append(weight_xgb)
    details.append(weight_lgb)
    details.append(weight_cab)
    details.append(weight_rf)
    details.append(ii)
    details.append(jj)
    details.append(kk)
    details.append(ll)

    cnt+=1
    print('每{}次验证的f1:{}'.format(cnt,max(scores)))
    mean_f1+=max(scores)/n_splits

    test_xgb = xlf.predict_proba(test_data)
    test_lgb = llf.predict_proba(test_data)
    test_cab = clf.predict_proba(test_data)
    test_rf = rf.predict_proba(test_data)
    #加权平均
    ans = test_xgb * ii + test_lgb * jj + test_cab * kk + test_rf*ll#加权平均
    #加权平方平均
    #ans = np.sqrt(test_xgb**2 * ii + test_lgb**2 * jj + test_cab**2 * kk + test_rf**2*ll)
    answers.append(ans)
print('mean f1:',mean_f1)

每1次验证的f1:0.8418367346938775
每2次验证的f1:0.8693467336683417
每3次验证的f1:0.85012285012285
每4次验证的f1:0.8480392156862746
每5次验证的f1:0.8512820512820514
mean f1: 0.8521255170906791


In [16]:
df=pd.DataFrame(np.array(details).reshape(int(len(details)/9),9)
                ,columns=['test_end_score','xgboost','lightgbm','catboost','rf'
                ,'weight_xgboost','weight_lightgbm','weight_catboost','weight_rf'])
df

,test_end_score,xgboost,lightgbm,catboost,rf,weight_xgboost,weight_lightgbm,weight_catboost,weight_rf
0,0.841837,0.827068,0.837563,0.840506,0.839695,0.1,0.1,0.1,0.7
1,0.869347,0.870000,0.848485,0.859296,0.861461,0.2,0.1,0.6,0.1
2,0.850123,0.843902,0.838235,0.841076,0.840796,0.2,0.1,0.1,0.6
3,0.848039,0.836186,0.835381,0.834568,0.847880,0.1,0.1,0.5,0.3
4,0.851282,0.824742,0.846939,0.851282,0.843188,0.1,0.1,0.6,0.2


In [17]:
fina=sum(answers)/n_splits#
#fina=np.sqrt(sum(np.array(answers)**2)/n_splits)#平方平均
fina=fina[:,1]

In [18]:
entprise_test = pd.read_csv('entprise_evaluate.csv')
dict1 = {
    'id':entprise_test.id,
    'score':fina
}
df = pd.DataFrame(dict1)
df[df['score']>=0.5]

,id,score
1,f000950527a6feb670cc1c87c2025f3922aaa4a0206a0a33,0.693468
37,82750f1b9d1223502cdde9ba782cad55e52842584fe34e78,0.794626
53,e9f7b28ec10e047089da56e946badaf8478af24af225838e,0.658691
56,59b38c56de383683fbf0391346de363a8c529780e7ba7156,0.721837
68,f000950527a6feb63fbd71bde689bf93d6ce72edd71e5d8d,0.903259
...,...,...
9930,59b38c56de383683eeb1d417fd4e036250ba48383b6d9512,0.806848
9961,82750f1b9d122350fe43715671f7fe9751ce0db3f171d192,0.524074
9972,59b38c56de383683ebfcfa6be70d1d44b2827a2f3f518dce,0.798033
9990,f000950527a6feb624e4b51b194d181ae1d48ef3e8b8964b,0.809823


In [19]:
entprise_test = pd.read_csv('entprise_evaluate.csv')
dict1 = {
    'id':entprise_test.id,
    'score':fina
}
df = pd.DataFrame(dict1)
df.to_csv(r'./result/1203_8.csv', index=False)